# Imputación básica

## Preparar datos

In [40]:
import pandas as pd
import numpy as n
import matplotlib.pyplot as plt
import random
import os
import warnings
import math

Hay que leer el dataset original para comparar los datos imputados con los reales

In [41]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
series = dataframe.squeeze('columns')
series = series.resample('D').mean()
series['1984-12-31'] = (series['1984-12-30'] + series['1985-01-01'])/2
series['1988-12-31'] = (series['1988-12-30'] + series['1989-01-01'])/2
series.head()

Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cogemos una lista con los datasets con los que vamos a trabajar

In [42]:
lista_datasets = os.listdir('..\miss_valued_data')

## Funciones auxiliares

### Calcular RMSE

In [43]:
def RMSE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += (serie_original[i] - serie_imputada[i])**2
    suma = math.sqrt(suma/len(serie_original))
    return suma

### Calcular MAE

In [50]:
def MAE(serie_original, serie_nula, serie_imputada):
    suma=0
    for i in serie_nula[serie_nula.isna()].index:
        suma += abs(serie_original[i] - serie_imputada[i])
    suma = suma/len(serie_original)
    return suma

## Imputamos

In [52]:
for i in lista_datasets:
    # Lo leemos
    df = pd.read_csv("..\miss_valued_data\\"+i, header=0, index_col=0, parse_dates=True)
    serie_nulo = df.squeeze('columns')
    
    # Imputamos con la media:
    serie_media = serie_nulo.fillna(serie_nulo.mean())
    # Imputamos con la mediana
    serie_mediana = serie_nulo.fillna(serie_nulo.median())

    # Calculamos la raiz del error cuadrático medio con las series imputadas con la media y la mediana
    rmse_media = RMSE(series, serie_nulo, serie_media)
    rmse_mediana = RMSE(series, serie_nulo, serie_mediana)
    # Calculamos el error medio absoluto con las series imputadas con la media y la mediana
    mae_media = MAE(series, serie_nulo, serie_media)
    mae_mediana = MAE(series, serie_nulo, serie_mediana)
    print("El RMSE imputando con la media el dataset " + i + " es : " + str(rmse_media))
    print("El RMSE imputando con la mediana el dataset " + i + " es : " + str(rmse_mediana))
    print("El MAE imputando con la media el dataset " + i + " es : " + str(mae_media))
    print("El MAE imputando con la mediana el dataset " + i + " es : " + str(mae_mediana))
    print("------------------------------------------------------------------------------------------------------------------")

El RMSE imputando con la media el dataset min_temperature_bateria_10.csv es : 1.4747579259701238
El RMSE imputando con la mediana el dataset min_temperature_bateria_10.csv es : 1.4694751449185541
El MAE imputando con la media el dataset min_temperature_bateria_10.csv es : 0.37715499689939913
El MAE imputando con la mediana el dataset min_temperature_bateria_10.csv es : 0.37487677984665946
------------------------------------------------------------------------------------------------------------------
El RMSE imputando con la media el dataset min_temperature_bateria_15.csv es : 1.6719544860161197
El RMSE imputando con la mediana el dataset min_temperature_bateria_15.csv es : 1.668904325721468
El MAE imputando con la media el dataset min_temperature_bateria_15.csv es : 0.5193103772137296
El MAE imputando con la mediana el dataset min_temperature_bateria_15.csv es : 0.5173466593647317
--------------------------------------------------------------------------------------------------------